# The Relation between the SEIR Epidemic Model and Hawkes Processes
> Recently, a link between the SIR compartmental epidemic model and Hawkes-like processes was found. This opens up the possibility to use some of the rich theory related to Hawkes processes when analyzing spreads of diseases using the SIR model. As the SIR model is not capable of modeling incubation times – a feature inherent in Covid-19 and many other diseases – there is a need to extend the recently found link to more complex compartmental epidemic models like the SEIR model. We have discovered this link and present it briefly in this blog post.
- toc: false
- branch: master
- badges: true
- comments: true
- categories: [epidemic models]
- hide: false
- search_exclude: true

In [1]:
#hide
import altair as alt

In [2]:
#hide
from sim_params import *

In [3]:
#hide
slider_beta = alt.binding_range(min=beta_min, max=beta_max, step=beta_step, name="\u03b2 ")
slider_sigma = alt.binding_range(min=gamma_min, max=gamma_max, step=gamma_step, name="\u03c3 ")
slider_gamma = alt.binding_range(min=gamma_min, max=gamma_max, step=gamma_step, name="\u03b3 ")

select_beta = alt.selection_single(
    fields=["beta"],
    bind={"beta": slider_beta},
    init={"beta": 1},
    name="parameter_beta"
)

select_gamma = alt.selection_single(
    fields=["gamma"],
    bind={"gamma": slider_gamma},
    init={"gamma": .1},
    name="parameter_gamma"
)

select_sigma = alt.selection_single(
    fields=["sigma"],
    bind={"sigma": slider_sigma},
    init={"sigma": .3},
    name="parameter_sigma"
)

In [4]:
#hide
sir_data = {"source": "https://raw.githubusercontent.com/yogabonito/test/master/_notebooks/plots/sir.json", "title": "SIR"}
seir_data = {"source": "https://raw.githubusercontent.com/yogabonito/test/master/_notebooks/plots/seir.json", "title": "SEIR"}
exp_hawkesn_data = {
    "source": "https://raw.githubusercontent.com/yogabonito/test/master/_notebooks/plots/exp_hawkesn.json",
    "title": "HawkesN process with exponential kernel"
}
hypoexp_and_erlang_hawkesn_data = {
    "source": "https://raw.githubusercontent.com/yogabonito/test/master/_notebooks/plots/hypoexp_and_erlang_hawkes.json",
    "title": "HawkesN process with hypoexponential/Erlang kernel"
}

sim_data = [sir_data, seir_data, exp_hawkesn_data, hypoexp_and_erlang_hawkesn_data]

In [44]:
#hide
sir_chart, seir_chart, exp_hawkesn_chart, hypoexp_and_erlang_hawkesn_chart = (
    alt.Chart(data["source"]).mark_bar().encode(
        alt.X(
            "day:Q",
            scale=alt.Scale(domain=(1, t_max-5))
        ),    
        alt.Y(
            "cumulative infections:Q",
            scale=alt.Scale(domain=(0, n))
        ),
        #x="day:Q"
        tooltip=["day:Q", "cumulative infections:Q"]
    ).properties(
        title=data["title"]
    )
    for data in sim_data
)

sir_chart, seir_chart, exp_hawkesn_chart, hypoexp_and_erlang_hawkesn_chart = (
    chart.add_selection(
        select_gamma
    ).transform_filter(
        select_gamma
    ).add_selection(
        select_beta
    ).transform_filter(
        select_beta
    )
    for chart in [sir_chart, seir_chart, exp_hawkesn_chart, hypoexp_and_erlang_hawkesn_chart]
)

seir_chart, hypoexp_and_erlang_hawkesn_chart = (
    chart.add_selection(
        select_sigma
    ).transform_filter(
        select_sigma
    )
    for chart in [seir_chart, hypoexp_and_erlang_hawkesn_chart]
)

In [45]:
#hide_input
(sir_chart | seir_chart) & (exp_hawkesn_chart | hypoexp_and_erlang_hawkesn_chart)

alt.VConcatChart(...)

## Introduction

Since their introduction in the 1970s, Hawkes processes have been successfully applied in many different fields, such as seismology, finance, and information science. Alan G. Hawkes also considered his class of stochastic processes "a possible epidemic model in large populations"{% fn 1 %}.

However, a model assuming an infinitely large population is often not suitable when modeling the spread of diseases. That's why many researchers preferred compartmental epidemic models (like the SIR or SEIR model) to Hawkes processes.

Only recently, it has been shown that there is a link between Hawkes-like processes and the SIR compartmental model.{% fn 2 %} As Rizoiu et al. put it, "[t]his is significant, as it
indicates that tools developed for one approach can be applied to the other." However, this link is restricted to the SIR compartmental model. As this model lacks the capability to model incubation time, many researchers turn to more complex models such as the SEIR model and variants thereof. This is especially true for diffusion analyses in the current Covid-19 crisis.

We have discovered a link between the SEIR model and Hawkes-like processes. In this blog post, we explain it and give a brief overview of all the models mentioned so far. For a more detailed derivation of the results and a more in-depth discussion please refer to our <a href="https://todo">paper</a>.

But before we dive into the details, let's get a feel of the models we will talk about. The plot at the beginning of this blog post shows four subplots, each containing the total number of infections (y-axis) that occured until a certain day (x-axis) in a population of 10 thousand people using one specific simulation model. When you hover over the bars, you will notice that many infection counts are not integers; that's because they are calculated as the mean of many simulations. Fiddle around with the sliders below the plot and see how they affect the spread of a disease. Comparing the subplots, we notice three things:

> Note: The two subplots on the left look similar. This is a consequence of the finding of Rizoiu et al. that we mentioned above: There is a link between the SIR model and Hawkes-like processes.

> Note: The two subplots on the right look similar. This is a consequence of our finding: There is a link between the SEIR model and Hawkes-like processes.

> Important: The subplots on the left and on the right are **very** different! Also note, that you can adjust the latency in the models plotted on the right (higher values for $\sigma$ implying less latency). The models on the left, however, are not capable of modeling latency (and thus incubation time). This underlines the importance of extending the link found by Rizoiu et al. to other compartmental models.

## Hawkes Processes

A Hawkes process is a type of counting process. A counting process counts events. These events can for example be infections (in epidemiology) or earthquakes (in seismology). Whenever an event occurs, the counting process experiences a jump of size $1$. Let's have a look at a sample path of a Hawkes process $N_t$ in the following figure (the blue line).

![Hawkes process - sample path and corresponding conditional intensity](./fig/hawkes_sample_path_with_conditional_intensity.svg)

You will notice that its jumps are not evenly distributed over the observed time interval $[0, 60]$. Why's that?

It's because the behavior of $N_t$ is dictated by its so-called conditional intensity $\lambda_t$ (the orange line). Whenever $\lambda_t$ is high we have a high probability of events. A low $\lambda_t$, on the other hand, leads to fewer jumps in $N_t$. This property is valid for all counting processes.

What makes Hawkes processes so special is that their conditional intensity $\lambda_t$ again depends on the counting process. In the figure above we see that the jumps in $N_t$ also increase the conditional intensity.

In general, the conditional intensity of a Hawkes process has the form
\begin{align}
\lambda_t
= \mu(t) + \sum\limits_{t_j < t} \nu(t-t_j)
,
\end{align}
where $t_j$ is an event time and $\mu(t)$ as well as $\nu(t-t_j)$ are both $\geq 0$.{% fn 3 %}
The function $\mu$ is called the background intensity. It is independent of $N_t$. The function $\nu$, however, depends on the history of the counting process $N_t$. We call $\nu$ the kernel of the Hawkes process.

In the figure above, $\mu(t)$ was independent of $t$ (hence we simply write $\mu$) and $\nu$ was an exponential kernel; this means
\begin{align}
\lambda_t
= \mu
+
\sum\limits_{t_j < t} \kappa \theta \text{e}^{-\theta (t - t_j)}.
\end{align}

Note that the exponential kernel leads to immediate jumps in $\lambda_t$ when an event occurs. When modeling infections, this means that an infection immediately increases the probability of further infections. This also means, that there is no way of taking incubation time into account. We will adress this issue below. But first, let's head to the other class of models we discuss in this post.

## Compartmental Epidemic Models

### The SIR Model

![SIR model](./fig/flowcharts/sir.svg)

### The SEIR Model

![SEIR model](./fig/flowcharts/seir.svg)

$\newcommand{\testnewcommand}{\text{newcommand works :)}}$

## The Link between the SIR & SEIR Model and the Hawkes Processes

### The SIR Model and the HawkesN Process with an Exponential Kernel

### The SEIR Model and the HawkesN Process with a Hypoexponential or Erlang Kernel

$\testnewcommand$

\newcommand{\testnewcommandinrawcell}{\text{newcommand works in raw cell :)}}

$\testnewcommandinrawcell$

## References

{{ 'Hawkes (1971): "Spectra of some self-exciting and mutually exciting point processes"' | fndetail: 1 }}

{{ 'Rizoiu et al. (2018): "SIR-Hawkes: Linking Epidemic Models and Hawkes Processes to Model Diffusions in Finite Populations"' | fndetail: 2 }}

{{ 'https://mathworld.wolfram.com/HawkesProcess.html' | fndetail: 3 }}